In [ ]:
import sympy as sp
import numpy as np

#Gleep glorp

def gaussian_error_propagation(formula, variables, result_lenght=4, output=True):
    """
    Berechnet die Fehlerfortpflanzung nach der Gaußschen Methode und gibt die
    verwendete Formel sowie das Ergebnis aus.
    
    Variablen werden in der Form (Name, Wert, Fehler) notiert bzw. (Name, Wert, 0) falls
    nicht nach ihnen abgeleitet werden soll.
    
    Parameter result_lenght passt die Rundungslänge des Ergebnisses an.
    """

    # create lists of variable names for later usage
    names = [var[0] for var in variables]
    err_names = [sp.symbols(f'del_{var[0]}') for var in variables]
    diff_functions, error_sums_n, error_sums_v =np.empty(0), np.empty(0), np.empty(0)
    val_dict={var[0]: var[1] for var in variables}
    
    # calculate formula value
    formula_value = formula.subs(val_dict).evalf()
    
    # differentiate function by each variable and make the sums both as strings as well as value
    i=0
    while i<len(names):
        diff_functions=np.append(diff_functions,sp.diff(formula, names[i]))
        if variables[i][2]!=0:
            error_sums_n=np.append(error_sums_n,diff_functions[i]*err_names[i])
        error_sums_v=np.append(error_sums_v,(diff_functions[i]*variables[i][2])**2)
        i+=1
    
    # create final formula as a string
    sum_str = [f'({str(sum_n)})**2' for sum_n in error_sums_n]
    result_str = ' + '.join(sum_str)
    error_formula=f'sqrt({result_str})'
    
    # calculate error value
    error_result=sp.sqrt(sum([expr.subs(val_dict).evalf() for expr in error_sums_v]))
    
    # print output
    if output==True:
        print(f'Formel: {formula}\nWerte: {variables} \n\nFormelwert: {formula_value}\n')
        print(f'Fehlerformel: {error_formula}')
        print(f'Fehler: {error_result} \nErgebnis: {round(formula_value,result_lenght)}±{round(error_result,result_lenght)}')
        return(None)
    else:
        return(formula_value, error_result)

In [75]:
# Beispielformel/variabeln
# notiere Variabelnamen durch kommata getrennt und dann nochmal in sp.symbols(' ')
# notiere dann Formel und verwende sp. für Funktionen wie sin()
# Trage zuletzt Werte der Variabeln als zweites Argument in Funktion: ((var_1,wert,err),(var_2,wert,err),...)

A_U, U_ges = sp.symbols('A_U U_ges')
calculated_formula = A_U/50*U_ges

gaussian_error_propagation(calculated_formula,((A_U, 1000, 10),(U_ges, 0.000853,0)))

Formel: A_U*U_ges/50
Werte: ((A_U, 1000, 10), (U_ges, 0.000853, 0)) 

Formelwert: 0.0170600000000000

Fehlerformel: sqrt((U_ges*del_A_U/50)**2)
Fehler: 0.000170600000000000 
Ergebnis: 0.0171±0.0002


In [76]:
# Beispielformel/variabeln
# notiere Variabelnamen durch kommata getrennt und dann nochmal in sp.symbols(' ')
# notiere dann Formel und verwende sp. für Funktionen wie sin()
# Trage zuletzt Werte der Variabeln als zweites Argument in Funktion: ((var_1,wert,err),(var_2,wert,err),...)

A_I, I_ges = sp.symbols('A_I I_ges')
calculated_formula = A_I/50*I_ges

gaussian_error_propagation(calculated_formula,((A_I, 1000, 10),(I_ges, 0.000853,0)))

Formel: A_I*I_ges/50
Werte: ((A_I, 1000, 10), (I_ges, 0.000853, 0)) 

Formelwert: 0.0170600000000000

Fehlerformel: sqrt((I_ges*del_A_I/50)**2)
Fehler: 0.000170600000000000 
Ergebnis: 0.0171±0.0002


In [77]:
import sympy as sp
import numpy as np
import os

def evaluate_gaussian_error(file_path, formulas, variables, result_names, result_length=4, feedback=True, output_file_suffix='results'):
    """
    Funktion zur automatischen Auswertung von Messdaten mit Gaußscher Fehlerfortpflanzung.
    Speichert Ergebnisse als Datei mit dem ursprünglichen Dateinamen plus dem angegebenen Suffix im selben Verzeichnis.
    
    Parameter:
    - file_path: Pfad zur Datei mit den Messdaten.
    - formulas: Liste von sympy-Formeln, auf die die Daten angewendet werden sollen.
    - variables: Liste der sympy-Variablennamen, z.B. [x, y, z].
    - result_names: Liste der Ergebnisvariablen für den Header der Ausgabedatei.
    - result_length: Anzahl der Dezimalstellen zur Rundung der Ergebnisse (default=4).
    - feedback: Wenn True, wird der Output für jede Zeile ausgegeben (default=True).
    - output_file_suffix: Suffix, das an den Dateinamen der Eingabedatei angehängt wird (default='results').
    """
    
    # Überprüfen, ob die Anzahl der Formeln und Ergebnisnamen übereinstimmt
    if len(formulas) != len(result_names):
        raise ValueError("Die Anzahl der Formeln und Ergebnisnamen muss übereinstimmen.")
    
    # Lade die Daten aus der Datei. Gehe davon aus, dass jede Variable eine Spalte für Werte und eine für Fehler hat
    data = np.loadtxt(file_path, usecols=range(len(variables) * 2), ndmin=1)
    
    # Initialisiere Ergebnisliste
    results = []
    
    l = len(data)
    num_formulas = len(formulas)
    
    # Variablen und ihre Fehler symbolisch als sympy Symbole
    err_vars = [sp.symbols(f'del_{var}') for var in variables]
    
    # Berechne die partiellen Ableitungen für alle Formeln vorab
    partial_derivatives_list = []
    for formula in formulas:
        partial_derivatives = [sp.diff(formula, var) for var in variables]
        partial_derivatives_list.append(partial_derivatives)
    
    # Iteriere über jede Zeile der Datei (jeder Zeile entsprechen Werte und Fehler für alle Variablen)
    for i in range(l):
        var_values = {}
        var_errors = {}
        
        # Extrahiere die Werte und Fehler für jede Variable aus der Datei
        for j, var in enumerate(variables):
            value = data[i, 2 * j]        # Wert
            error = data[i, 2 * j + 1]    # Fehler
            var_values[var] = value
            var_errors[var] = error
        
        row_results = []
        
        # Wende die Gaußsche Fehlerfortpflanzung für jede Formel an
        for k in range(num_formulas):
            formula = formulas[k]
            partial_derivatives = partial_derivatives_list[k]
            # Berechne den Funktionswert
            func_value = formula.evalf(subs=var_values)
            # Berechne den Fehler nach der Gaußschen Fehlerfortpflanzung
            squared_errors = []
            for pd, var in zip(partial_derivatives, variables):
                pd_value = pd.evalf(subs=var_values)
                error = var_errors[var]
                squared_errors.append((pd_value * error) ** 2)
            error_value = sp.sqrt(sum(squared_errors)).evalf()
            # Runde das Ergebnis
            func_value = round(float(func_value), result_length)
            error_value = round(float(error_value), result_length)
            # Füge das Ergebnis der Zeile hinzu
            row_results.extend([func_value, error_value])
        
        results.append(row_results)
        
        # Ausgabe für den Nutzer (optional)
        if feedback:
            print(f"Zeile {i+1}: {row_results}")
    
    # Ermittle den Ordner und den Dateinamen ohne die Dateiendung der ursprünglichen Datei
    folder_path = os.path.dirname(file_path)
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Erstelle den Ausgabedateinamen durch Anhängen des Suffixes
    output_file_name = f"{base_name}_{output_file_suffix}.txt"
    output_file_path = os.path.join(folder_path, output_file_name)
    
    # Erstelle den Header für die Ausgabedatei
    header_items = []
    for name in result_names:
        header_items.extend([name, f"err_{name}"])
    # Anpassung: Keine Leerzeichen nach '#' und Leerzeichen als Trennzeichen
    header = '#' + ' '.join(header_items)
    
    # Speichere die Ergebnisse in der Datei im gleichen Ordner wie die ursprüngliche Datei
    np.savetxt(output_file_path, results, fmt=f'%.{result_length}f', header=header, delimiter=' ')
    
    # Gib eine Erfolgsnachricht aus
    print(f"Auswertung abgeschlossen. Ergebnisse wurden in '{output_file_path}' gespeichert.")


In [78]:
import sympy as sp

#Ueff err_U_eff C errC Omega errOmega
# Definiere die Variablen

Skt, syst_err, dist, R, I, R1, R2, U0  = sp.symbols("Skt syst_err dist R I R1 R2 U0 ")

var_names = [Skt, syst_err, dist, R, I, R1, R2, U0]

# Definiere die Formeln
formulas = [
    0.5 * ((Skt * 0.001) / dist) * syst_err,
    R,
    I,
    R1,
    R2,
    U0
]

# Definiere die Ergebnisnamen
result_names = ['Phi', 'R', 'I',  'R1', 'R2', 'U0']

# Pfad zur Eingabedatei
file_path = '236/c.txt'

# Aufruf der Funktion mit dem gewünschten Suffix
evaluate_gaussian_error(
    file_path,
    formulas,
    var_names,
    result_names,
    feedback=True,
    output_file_suffix='rech',
    result_length=7
)


Zeile 1: [0.207, 0.0231435, 200.6, 0.3, 0.00022, 2e-06, 100.7, 0.3, 3.1, 0.1, 2.64, 0.2]
Zeile 2: [0.178, 0.0199013, 250.7, 0.3, 0.000193, 2e-06, 100.7, 0.3, 3.1, 0.1, 2.64, 0.2]
Zeile 3: [0.157, 0.0175534, 300.8, 0.3, 0.000173, 2e-06, 100.7, 0.3, 3.1, 0.1, 2.64, 0.2]
Zeile 4: [0.139, 0.015541, 350.9, 0.3, 0.000155, 2e-06, 100.7, 0.3, 3.1, 0.1, 2.64, 0.2]
Zeile 5: [0.097, 0.0108454, 553.2, 0.3, 0.000111, 2e-06, 100.7, 0.3, 3.1, 0.1, 2.64, 0.2]
Zeile 6: [0.0785, 0.0087771, 702.0, 0.3, 9.2e-05, 2e-06, 100.7, 0.3, 3.1, 0.1, 2.64, 0.2]
Auswertung abgeschlossen. Ergebnisse wurden in '236\c_rech.txt' gespeichert.


In [79]:
import sympy as sp

#Ueff err_U_eff C errC Omega errOmega
# Definiere die Variablen

Phi, R, I, R1, R2, U0  = sp.symbols("Phi R I R1 R2 U0 ")

var_names = [Phi,R, I, R1, R2, U0]

# Definiere die Formeln
formulas = [
    (1/(Phi+0.01)) /((R1 + R2)/(U0 * R2)),
    R
]

# Definiere die Ergebnisnamen
result_names = ['Phi_-1', 'R_mit_Faktor']

# Pfad zur Eingabedatei
file_path = '236/c_rech.txt'

# Aufruf der Funktion mit dem gewünschten Suffix
evaluate_gaussian_error(
    file_path,
    formulas,
    var_names,
    result_names,
    feedback=True,
    output_file_suffix='R_gg_Phi-1_mit_Faktor',
    result_length=7
)


Zeile 1: [0.3633361, 0.048884, 200.6, 0.3]
Zeile 2: [0.4193826, 0.056161, 250.7, 0.3]
Zeile 3: [0.4721193, 0.0629445, 300.8, 0.3]
Zeile 4: [0.5291539, 0.0702118, 350.9, 0.3]
Zeile 5: [0.7368592, 0.0960762, 553.2, 0.3]
Zeile 6: [0.8908919, 0.1146549, 702.0, 0.3]
Auswertung abgeschlossen. Ergebnisse wurden in '236\c_rech_R_gg_Phi-1_mit_Faktor.txt' gespeichert.


In [80]:
import sympy as sp

#Ueff err_U_eff C errC Omega errOmega
# Definiere die Variablen

Phi, R, I, R1, R2, U0  = sp.symbols("Phi R I R1 R2 U0")

var_names = [Phi, R, I, R1, R2, U0]

# Definiere die Formeln
formulas = [
    I,
    Phi + 0.01,
]

# Definiere die Ergebnisnamen
result_names = ['I', 'Phi']

# Pfad zur Eingabedatei
file_path = '236/c_rech.txt'

# Aufruf der Funktion mit dem gewünschten Suffix
evaluate_gaussian_error(
    file_path,
    formulas,
    var_names,
    result_names,
    feedback=True,
    output_file_suffix='Phi_gg_I',
    result_length=7
)


Zeile 1: [0.00022, 2e-06, 0.217, 0.0231435]
Zeile 2: [0.000193, 2e-06, 0.188, 0.0199013]
Zeile 3: [0.000173, 2e-06, 0.167, 0.0175534]
Zeile 4: [0.000155, 2e-06, 0.149, 0.015541]
Zeile 5: [0.000111, 2e-06, 0.107, 0.0108454]
Zeile 6: [9.2e-05, 2e-06, 0.0885, 0.0087771]
Auswertung abgeschlossen. Ergebnisse wurden in '236\c_rech_Phi_gg_I.txt' gespeichert.


In [81]:
import sympy as sp

#Ueff err_U_eff C errC Omega errOmega
# Definiere die Variablen

t, Skt, syst_err, dist = sp.symbols("t Skt syst_err dist")

var_names = [t, Skt, syst_err, dist]

# Definiere die Formeln
formulas = [
    t,
    (Skt + 5) * syst_err #(((Skt+5) * 0.001) / dist) * syst_err 
]

# Definiere die Ergebnisnamen
result_names = ['t', 'phi']

# Pfad zur Eingabedatei
file_path = '236/i.txt'

# Aufruf der Funktion mit dem gewünschten Suffix
evaluate_gaussian_error(
    file_path,
    formulas,
    var_names,
    result_names,
    feedback=True,
    output_file_suffix='rech',
    result_length=7
)


Zeile 1: [1.25, 0.1, 41.0, 3.6335245]
Zeile 2: [1.64, 0.1, 27.0, 3.2897568]
Zeile 3: [1.95, 0.1, 22.0, 3.1953091]
Zeile 4: [2.22, 0.1, 17.0, 3.1180924]
Zeile 5: [2.84, 0.1, 11.0, 3.05]
Zeile 6: [1.8, 0.1, 25.0, 3.25]
Zeile 7: [1.66, 0.1, 28.0, 3.3105891]
Zeile 8: [1.67, 0.1, 28.0, 3.3105891]
Zeile 9: [2.1, 0.1, 18.0, 3.132092]
Auswertung abgeschlossen. Ergebnisse wurden in '236\i_rech.txt' gespeichert.


In [82]:
import sympy as sp

#Ueff err_U_eff C errC Omega errOmega
# Definiere die Variablen

t, Phi = sp.symbols("t Phi")

var_names = [t, Phi]

# Definiere die Formeln
formulas = [
    t,
    sp.log(Phi),
]

# Definiere die Ergebnisnamen
result_names = ['t', 'log(phi)']

# Pfad zur Eingabedatei
file_path = '236/i_rech.txt'

# Aufruf der Funktion mit dem gewünschten Suffix
evaluate_gaussian_error(
    file_path,
    formulas,
    var_names,
    result_names,
    feedback=True,
    output_file_suffix='phi_gg_log_t',
    result_length=7
)


Zeile 1: [1.25, 0.1, 3.7135721, 0.0886225]
Zeile 2: [1.64, 0.1, 3.2958369, 0.1218428]
Zeile 3: [1.95, 0.1, 3.0910425, 0.1452413]
Zeile 4: [2.22, 0.1, 2.8332133, 0.1834172]
Zeile 5: [2.84, 0.1, 2.3978953, 0.2772727]
Zeile 6: [1.8, 0.1, 3.2188758, 0.13]
Zeile 7: [1.66, 0.1, 3.3322045, 0.1182353]
Zeile 8: [1.67, 0.1, 3.3322045, 0.1182353]
Zeile 9: [2.1, 0.1, 2.8903718, 0.1740051]
Auswertung abgeschlossen. Ergebnisse wurden in '236\i_rech_phi_gg_log_t.txt' gespeichert.
